<a href="https://colab.research.google.com/github/dipnarayan501/Deep_learning/blob/main/Assignment1_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Importing libraries

import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, fashion_mnist
from sklearn.model_selection import train_test_split

Download the fashion-MNIST dataset and plot 1 sample image for each class as shown in the grid below. Use "from keras.datasets import fashion_mnist" for getting the fashion mnist dataset.

In [11]:
# Preparing Dataset

def prepare_dataset():
    (X, y), (X_test, y_test) = fashion_mnist.load_data()

    # Reshaping the data matrices
    X = X.reshape(X.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Normalizing the pixel intensities
    X = X/255.0
    X_test = X_test/255.0
    
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    
    y_train_encode = np.zeros((10,y_train.shape[0]))
    y_train_encode[y_train, np.array(list(range(y_train.shape[0])))] = 1

    # Split the X_train into a training set and validation set
    
    
    print("Number of examples in training set")
    print(X_train.shape)
    print(y_train_encode.shape)
    print(y_train.shape)
    
    print("No of examples in validation set")
    print(X_val.shape)
    print(y_val.shape)
    print("No of examples in test set")
    print(X_test.shape)
    print(y_test.shape)
    
    return X_train.T, y_train_encode, y_train.T ,X_val.T, y_val.T, X_test.T, y_test.T


X_train, y_train_encode, y_train ,X_val, y_val, X_test, y_test = prepare_dataset()

Number of examples in training set
(54000, 784)
(10, 54000)
(54000,)
No of examples in validation set
(6000, 784)
(6000,)
No of examples in test set
(10000, 784)
(10000,)


In [12]:
#Activation functions

def sigmoid(x):
    return 1. / (1.+np.exp(-x))
def relu(x):
    return np.maximum(0,x)
def tanh(x):
    return np.tanh(x)
def softmax(x):
    soft = np.zeros(x.shape)
    for i in range(0, x.shape[1]):
        numr = np.exp(x[:, i])
        soft[:, i] = numr/np.sum(numr)
    return soft


def sigmoid_derivative(x):
    return sigmoid(x) * (1-sigmoid(x))
def relu_derivative(x):
    return 1*(x>0)
def tanh_derivative(x):
    return (1 - (np.tanh(x)**2))



In [13]:
#Initializing parameters
output_layer = 10
input_layer = X_test.shape[0]
hidden_layer = [64, 64]
#Example  28*28,[64,64],output_classes
def initialize_parameters(input_layer, hidden_layer, output_layer,mode):
    weights = []
    biases  = []
    layers = [input_layer] + hidden_layer + [output_layer]
    for i in range(len(hidden_layer)+1):
        if mode == 'random_normal':
            weights.append(np.random.randn(layers[i+1], layers[i])) * 0.01
        if mode == 'xavier':
            weights.append(np.random.randn(layers[i+1],layers[i])*np.sqrt(2/layers[i+1]))
        biases.append(np.random.randn(layers[i+1],1))
    return weights, biases
weights, biases = initialize_parameters(input_layer, hidden_layer, output_layer, 'xavier')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)


(64, 784)
(64, 64)
(10, 64)
(64, 1)
(64, 1)
(10, 1)


In [14]:
#Forward Propogation
def forward_propagation(X, weights, biases,activation):

        A = []  #pre-activation
       
        H  = []    # post activation
        W      = weights
        b      = biases

        k=0
        A.append(np.dot(W[k],X)+b[k])
        if activation == 'sigmoid':
          H.append(sigmoid(A[k]))
        elif activation == 'tanh':
          H.append(tanh(A[k]))
        elif activation == 'relu':
          H.append(relu(A[k]))
        #elif self.activation == 'leaky_relu':
         # H.append(leaky_relu(A[k]))

        for k in range(1,len(hidden_layer)):
            A.append(np.dot(W[k],H[k-1])+b[k])
            if activation == 'sigmoid':
              H.append(sigmoid(A[k]))
            elif activation == 'tanh':
              H.append(tanh(A[k]))
            elif activation == 'relu':
              H.append(relu(A[k]))
            elif activation == 'leaky_relu':
              H.append(leaky_relu(A[k]))

        k=len(hidden_layer)
        A.append(np.dot(W[k],H[k-1])+b[k])
        H.append(softmax(A[k]))

        return A , H

   
A, H = forward_propagation(X_train, weights, biases, 'sigmoid')
for i in range(len(A)):
    print(A[i].shape)
   
for i in range(len(H)):
    print(H[i].shape)


(64, 54000)
(64, 54000)
(10, 54000)
(64, 54000)
(64, 54000)
(10, 54000)


In [15]:
#Backward propogation
def back_propagation(X_train,y_train,weights,biases,A ,H, activation ,loss_function):
    #Initialzing gradients
    g_weights = [0]*(len(weights))
    g_biases  = [0]*(len(biases))
    g_a       = [0]*(len(hidden_layer)+1)
    g_h       = [0]*(len(hidden_layer)+1)
    n_samples = X_train.shape[0]  # Change depending on the dimensions of data


    for k in range(len(hidden_layer),-1,-1): #2,1,0
        
        if k == len(hidden_layer):
            if loss_function == 'cross_entropy':
                g_a[k] = H[k]  - y_train  # keep or remove T depending on the dimensions of data
            elif loss_function == 'square_loss': 
                g_a[k] = (H[k] - y_train) * H[k] * (1 - H[k])
            
       
        else:
            g_h[k] = (1/n_samples)*np.dot(weights[k+1].T,g_a[k+1])
            if activation == 'sigmoid':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],sigmoid_derivative(A[k]))
            elif self.activation == 'tanh':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],tanh_derivative(A[k]))
            elif  activation == 'relu':
                g_a[k] = (1/n_samples)*np.multiply(g_h[k],relu_derivative(A[k]))


        if k == 0:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],X_train.T)
        else:
            g_weights[k] = (1/n_samples)*np.dot(g_a[k],H[k-1].T)

        g_biases[k]  = (1/n_samples)*np.sum(g_a[k], axis=1, keepdims = True)
    return g_weights,g_biases

g_weights,g_biases = back_propagation(X_train,y_train_encode,weights,biases,A,H,'sigmoid', 'square_loss')
for i in range(len(g_weights)):
    print(g_weights[i].shape)
    
for i in range(len(g_biases)):
    print(g_biases[i].shape)


(64, 784)
(64, 64)
(10, 64)
(64, 1)
(64, 1)
(10, 1)
